Practice python project for Data Enginnering

In [70]:
## Import libraries

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3
import requests

In [71]:
## Function for data extraction

def extract(url, table_attribis):
    "extracts the required information from the website and saves it to a dataframe."

    #Extract the webpage as text
    webpage = requests.get(url).text

    #Parse the text into HTML  Object
    data = BeautifulSoup(webpage, 'html.parser')

    #Create empty data frame
    df = pd.DataFrame(columns=table_attribis)

    #Extract tables from the data 
    tables = data.find_all("tbody")

    #Extract the rows for corresponding table
    rows = tables[2].find_all("tr")

    #Insert table data into DF
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            if col[0].find("a") is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0], # Get the first content inside the <a> tag
                            "GDP_USD_millions": col[2].contents[0]}
                df_temp = pd.DataFrame(data_dict, index = [0])
                df = pd.concat([df, df_temp], ignore_index=True)

    return df

In [72]:
## Function for data transformation

def transform(df):
    "convert GDP information from currrency format to float values. convert USD(millions) to USD(Billion) rounding to 2 decimal places"

    #Convert USD million to billion
    l = df['GDP_USD_millions'].to_list()
    l = [ np.round(float("".join(x.split(',')))/1000,2) for x in l ]

    df['GDP_USD_millions'] = l
    df.rename(columns={'GDP_USD_millions':"GDP_USD_billion"}, inplace=True)
    return df


In [73]:
# Function for load data into csv

def load_csv(df, csv_path):
    "Save the data in CSV format"
    df.to_csv(csv_path)
             

In [74]:
# Function for load data into db

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False) #Prevents writing the DataFrame index as a separate column.


In [75]:
# Function to run queries

def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [76]:
# function to take logs

def log_progress(message, log_file_path):
    time_stamp_format = "%Y-%m-%d-%H:%M:%S"
    now = datetime.now()
    time_stamp = now.strftime(time_stamp_format)

    with open(log_file_path, 'a') as file:
        file.write(time_stamp + " : " + message + "\n")

In [77]:
## Initiate parameters

url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ['Country', 'GDP_USD_millions']
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path  = './Countries_by_GDP.csv'
log_file_path = './etl_project_log.txt'

In [78]:
##Call for the data_extraction function

log_progress("Preliminaries complete. Initiating ETL process", log_file_path)

df = extract(url, table_attribs)
df

,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151


In [79]:
## transform the data

log_progress("Data extraction complete. Initiating Transformation process", log_file_path)

df = transform(df)
df

,Country,GDP_USD_billion
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88
...,...,...
186,Marshall Islands,0.29
187,Palau,0.26
188,Kiribati,0.25
189,Nauru,0.15


In [81]:
## Load the data into csv file

log_progress('Data transformation complete. Initiating loading process', log_file_path)
load_csv(df, csv_path)

log_progress('Data saved to CSV file', log_file_path)

In [82]:
#Establish the db connection
sql_connection  = sqlite3.connect(db_name)

log_progress('SQL Connection initiated.', log_file_path)

load_to_db(df, sql_connection, table_name)

In [83]:
## Run queries
query_statement = 'SELECT * FROM {} LIMIT 10;'.format(table_name)

log_progress('Data loaded to Database as table. Running the query', log_file_path)

run_query(query_statement, sql_connection)


log_progress('Process Complete.', log_file_path)

SELECT * FROM Countries_by_GDP LIMIT 10;
          Country  GDP_USD_billion
0   United States         26854.60
1           China         19373.59
2           Japan          4409.74
3         Germany          4308.85
4           India          3736.88
5  United Kingdom          3158.94
6          France          2923.49
7           Italy          2169.74
8          Canada          2089.67
9          Brazil          2081.24


In [84]:
#Remove the DB Connection
sql_connection.close()